# DAQL for BipedalWalker

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym
env = gym.make('BipedalWalker-v2')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.observation_space, env.action_space

(Box(24,), Box(4,))

In [4]:
# state = env.reset()
# batch = []
# for _ in range(1111):
#     #env.render()
#     action = env.action_space.sample()
#     next_state, reward, done, _ = env.step(action) # take a random action
#     batch.append([state, action, next_state, reward, float(done)])
#     state = next_state
#     if done:
#         state = env.reset()

To shut the window showing the simulation, use `env.close()`.

In [5]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [6]:
# import numpy as np
# actions = np.array([each[0] for each in batch])
# states = np.array([each[1] for each in batch])
# rewards = np.array([each[2] for each in batch])
# dones = np.array([each[3] for each in batch])
# infos = np.array([each[4] for each in batch])

In [7]:
# print('states:', np.max(np.array(states)), np.min(np.array(states)))
# print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

In [8]:
# env.action_space.high, env.action_space.low

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [9]:
def model_input(state_size, action_size):
    #states = tf.placeholder(tf.float32, [None, *state_shape], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.float32, [None, action_size], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_state')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    return states, actions, next_states, rewards, dones

In [10]:
# a = act(s)
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False, trainable=True):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size, trainable=trainable)
        bn1 = tf.layers.batch_normalization(h1, training=training, trainable=trainable)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size, trainable=trainable)
        bn2 = tf.layers.batch_normalization(h2, training=training, trainable=trainable)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size, trainable=trainable)        
        pred = tf.tanh(logits)
        return pred

In [11]:
# s', Q = env(s, a)
def discriminator(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False, 
                  trainable=True):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size, trainable=trainable)
        bn1 = tf.layers.batch_normalization(h1, training=training, trainable=trainable)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=fused, units=hidden_size, trainable=trainable)
        bn2 = tf.layers.batch_normalization(h2, training=training, trainable=trainable)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        next_states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=trainable)        
        Qlogits = tf.layers.dense(inputs=next_states_logits, units=1, trainable=False)        
        return next_states_logits, Qlogits

In [27]:
def model_loss(action_size, hidden_size, state_size, gamma,
               states, actions, next_states, rewards, dones):
    #########################################
    actions_pred = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    gen_next_states, gQlogits = discriminator(actions=actions_pred, states=states, hidden_size=hidden_size, 
                                         action_size=action_size, state_size=state_size)
    gQs = tf.reshape(gQlogits, shape=[-1])
    print('gQlogits.shape, gQs.shape:', gQlogits.shape, gQs.shape)
    gloss = -tf.reduce_mean(gQs)
    print('gen_next_states.shape, next_states.shape:', gen_next_states.shape, next_states.shape)
    gloss += -tf.reduce_mean(tf.reduce_sum(tf.square(gen_next_states - next_states), axis=1))
    ########################################
    next_actions_pred = generator(states=next_states, hidden_size=hidden_size, action_size=action_size, 
                                  trainable=False, reuse=True)
    _, nextQlogits = discriminator(actions=next_actions_pred, states=next_states, hidden_size=hidden_size,
                              action_size=action_size, state_size=state_size, 
                              trainable=False, reuse=True)
    nextQs = tf.reshape(nextQlogits, shape=[-1])
    targetQs = gamma * nextQs * (1-dones)
    print('nextQlogits.shape, nextQs.shape, targetQs.shape:', nextQlogits.shape, nextQs.shape, targetQs.shape)
    ###########################################
    next_states_pred, Qlogits = discriminator(actions=actions, states=states, hidden_size=hidden_size, 
                                         action_size=action_size, state_size=state_size, reuse=True)
    Qs = tf.reshape(Qlogits, shape=[-1])
    print('Qs.shape, Qlogits.shape:', Qs.shape, Qlogits.shape)
    rewards_in = Qs - targetQs
    print('rewards_in.shape, rewards.shape:', rewards_in.shape, rewards.shape)
    dloss = tf.reduce_mean(tf.square(rewards_in - rewards))
    print('pred_next_states.shape, next_states.shape:', next_states_pred.shape, next_states.shape)
    dloss += tf.reduce_mean(tf.reduce_sum(tf.square(next_states_pred - next_states), axis=1))
    return actions_pred, gloss, dloss

In [28]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, g_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
    return g_opt, d_opt

In [29]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, g_learning_rate, d_learning_rate, gamma):
        
        # Model inputs
        self.states, self.actions, self.next_states, self.rewards, self.dones = model_input(
            state_size=state_size, action_size=action_size)

        # Model loss/objective
        self.actions_pred, self.g_loss, self.d_loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, state_size=state_size, gamma=gamma,
            states=self.states, actions=self.actions, next_states=self.next_states,
            rewards=self.rewards, dones=self.dones)
        
        # Model optimization/update
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss,
                                           g_learning_rate=g_learning_rate, d_learning_rate=d_learning_rate)

In [30]:
import numpy as np
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [31]:
env.observation_space, env.action_space

(Box(24,), Box(4,))

In [32]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 24
action_size = 4
hidden_size = 24*2             # number of units in each Q-network hidden layer
g_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e2)             # experience mini-batch size == one episode size is 1000/int(1e3) steps
gamma = 0.99                   # future reward discount

In [33]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              g_learning_rate=g_learning_rate, d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

gQlogits.shape, gQs.shape: (?, 1) (?,)
gen_next_states.shape, next_states.shape: (?, 24) (?, 24)
nextQlogits.shape, nextQs.shape, targetQs.shape: (?, 1) (?,) (?,)
Qs.shape, Qlogits.shape: (?,) (?, 1)
rewards_in.shape, rewards.shape: (?,) (?,)
pred_next_states.shape, next_states.shape: (?, 24) (?, 24)


In [25]:
env.observation_space.high, env.observation_space.low, env.observation_space, \
env.action_space.high, env.action_space.low, env.action_space, \
env.reward_range

(array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       dtype=float32),
 array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf], dtype=float32),
 Box(24,),
 array([1., 1., 1., 1.], dtype=float32),
 array([-1., -1., -1., -1.], dtype=float32),
 Box(4,),
 (-inf, inf))

In [34]:
state = env.reset()

for each_step in range(memory_size):
    #env.render()

    action = env.action_space.sample() # randomness
    action = np.clip(action, -1, 1) # clipped: [-1, +1]
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    
    if done is True:
        print('Progress:', each_step/memory_size)
        state = env.reset()

Progress: 0.00073
Progress: 0.0013
Progress: 0.00178
Progress: 0.00277
Progress: 0.00331
Progress: 0.00487
Progress: 0.0055
Progress: 0.0215
Progress: 0.02222
Progress: 0.02268
Progress: 0.03868
Progress: 0.05468
Progress: 0.05538
Progress: 0.05589
Progress: 0.05684
Progress: 0.07284
Progress: 0.07358
Progress: 0.07419
Progress: 0.07482
Progress: 0.09082
Progress: 0.09199
Progress: 0.09257
Progress: 0.09352
Progress: 0.09411
Progress: 0.09493
Progress: 0.11093
Progress: 0.11169
Progress: 0.11243
Progress: 0.11306
Progress: 0.11383
Progress: 0.11472
Progress: 0.13072
Progress: 0.14672
Progress: 0.14726
Progress: 0.14777
Progress: 0.14869
Progress: 0.14954
Progress: 0.16554
Progress: 0.16659
Progress: 0.18259
Progress: 0.1839
Progress: 0.1999
Progress: 0.20069
Progress: 0.20139
Progress: 0.20263
Progress: 0.20344
Progress: 0.20419
Progress: 0.20514
Progress: 0.22114
Progress: 0.22187
Progress: 0.22313
Progress: 0.22372
Progress: 0.22439
Progress: 0.22502
Progress: 0.24102
Progress: 0.241

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [35]:
len(memory.buffer), memory.buffer.maxlen

(100000, 100000)

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, gloss_list, dloss_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100)
    
    # Training episodes/epochs
    for ep in range(11111):
        gloss_batch, dloss_batch = [], []
        total_reward = 0
        state = env.reset()

        # Training steps/batches
        while True:
            env.render()
            
            action_pred = sess.run(model.actions_pred, feed_dict={model.states: state.reshape([1, -1])})
            noise = np.random.normal(loc=0, scale=0.1, size=action_size) # randomness
            action = action_pred.reshape([-1]) + noise
            #print(action.shape, action_logits.shape, noise.shape)
            action = np.clip(action, -1, 1) # clipped
            
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            total_reward += reward
            state = next_state
            
            # Training
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            feed_dict = {model.states: states, 
                         model.actions: actions,
                         model.next_states: next_states, 
                         model.rewards: rewards, 
                         model.dones: dones}
            dloss, _ = sess.run([model.d_loss, model.d_opt], feed_dict)
            gloss, _ = sess.run([model.g_loss, model.g_opt], feed_dict)
            gloss_batch.append(gloss)
            dloss_batch.append(dloss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        gloss_list.append([ep, np.mean(gloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        # Break episode/epoch loop
        # Did not solve the environment. 
        # Best 100-episode average reward was 220.62 ± 0.69. 
        # (BipedalWalker-v2 is considered "solved" 
        #  when the agent obtains an average reward of at least 300 over 100 consecutive episodes.)        
        if np.mean(episode_reward) >= 300:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:-95.3386 R:-95.3386 gloss:-9.5685 dloss:24.7268
Episode:1 meanR:-100.2504 R:-105.1622 gloss:-9.2401 dloss:15.4197
Episode:2 meanR:-103.3185 R:-109.4548 gloss:-8.4681 dloss:19.8620
Episode:3 meanR:-107.6432 R:-120.6173 gloss:-7.8119 dloss:24.2005
Episode:4 meanR:-110.5241 R:-122.0479 gloss:-7.2402 dloss:15.2813
Episode:5 meanR:-112.5578 R:-122.7260 gloss:-6.7697 dloss:17.5413
Episode:6 meanR:-113.9808 R:-122.5191 gloss:-6.3166 dloss:10.0933
Episode:7 meanR:-114.9942 R:-122.0876 gloss:-5.8769 dloss:16.6124
Episode:8 meanR:-115.7929 R:-122.1824 gloss:-5.4707 dloss:9.4658
Episode:9 meanR:-116.5009 R:-122.8733 gloss:-5.0899 dloss:16.2435
Episode:10 meanR:-117.0499 R:-122.5393 gloss:-4.7606 dloss:9.0405
Episode:11 meanR:-117.4783 R:-122.1912 gloss:-4.4109 dloss:20.3934
Episode:12 meanR:-117.8748 R:-122.6323 gloss:-4.1419 dloss:3.6884
Episode:13 meanR:-118.2249 R:-122.7768 gloss:-3.9048 dloss:10.5443
Episode:14 meanR:-118.4909 R:-122.2147 gloss:-3.6164 dloss:5.5777
Episode:15 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [31]:
import gym
env = gym.make('BipedalWalker-v2')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model2.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            actions_pred = sess.run(model.actions_pred, feed_dict={model.states: state.reshape([1, -1])})
            action = action_pred.reshape([-1])
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: -104.69566602519279


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.